## train the LM 

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install https://github.com/kpu/kenlm/archive/master.zip 
!{sys.executable} -m pip install pyctcdecode==0.3.0
!{sys.executable} -m pip install datasets==2.0.0
!{sys.executable} -m pip install transformers==4.18.0

In [ ]:
!{sys.executable} -m pip install ipywidgets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev


In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz


In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2

In [ ]:
!ls kenlm/build/bin

In [ ]:
target_lang="sv"  # change to your target lang


In [ ]:
from datasets import load_dataset

username = "hf-test"  # change to your username

dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))


In [ ]:
!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"


In [ ]:
!head -20 5gram.arpa


In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
!head -20 5gram_correct.arpa

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("/home/ubuntu/LM/lingala_xlsr4/wav2vec2-large-xlsr-lingala")

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [ ]:
sorted_vocab_dict

In [ ]:
#from pyctcdecode import build_ctcdecoder


#labels = [
#    "s","b","l","u","ç","i","o","f","d","k","ɔ","t","r","w","j","p","x","v","c","z",
#    "h","q","y","m","|","e","n","ε","a","g","[UNK]","[PAD]","<s>","</s>"
#]
    
# prepare decoder and decode logits via shallow fusion
#decoder = build_ctcdecoder(
#    labels,
#    kenlm_model_path="5gram_correct.arpa",  # either .arpa or .bin file
#)

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/home/ubuntu/LM/lingala_xlsr4/wav2vec2-large-xlsr-lingala", eos_token=None, bos_token=None)

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="xlsr-lingala2", clone_from="Ussen/wav2vec2-large-xlsr-lingala")

In [ ]:
processor_with_lm.save_pretrained("xlsr-lingala2")

In [ ]:
#use the terminal
!kenlm/build/bin/build_binary xlsr-lingala2/language_model/5gram_correct.arpa xlsr-lingala2/language_model/5gram.bin

In [ ]:
!rm xlsr-lingala2/language_model/5gram_correct.arpa && tree -h xlsr-lingala2/


## push to hub

In [ ]:
!git clone https://huggingface.co/Ussen/xlsr-lingala2

In [ ]:
# this code worked with the following torch setup
!{sys.executable} -m pip install torch==1.12.0+cu116 torchvision==0.13.0+cu116 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC

processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/ubuntu/LM/xlsr-lingala2")
model = Wav2Vec2ForCTC.from_pretrained("/home/ubuntu/LM/xlsr-lingala2")

In [ ]:
from transformers import AutoProcessor

processor1 = AutoProcessor.from_pretrained("/home/ubuntu/LM/lingala_xlsr4/wav2vec2-large-xlsr-lingala")

## load the test set

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install torchaudio
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install jiwer
!{sys.executable} -m pip install -qqq evaluate==0.2.2

In [ ]:
import os
#Connect to Gdrive to store model checkpoints
#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)

In [ ]:
import numpy as np
import zipfile
import csv

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/home/ubuntu/LM/LingalaAudio-20221111T081505Z-001.zip", 'r')
zip_ref.extractall("/home/ubuntu/LM")
zip_ref.close()

In [ ]:
with open('/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/train.csv', newline='',encoding='UTF-8') as f:
      reader = csv.reader(f)
      data = list(reader)
      train_data = [data[i] for i in range(len(data)) if i!=0]

with open('/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/test.csv', newline='',encoding='UTF-8') as f:
      reader = csv.reader(f)
      data = list(reader)
      t_data = [data[i] for i in range(len(data)) if i!=0]

In [ ]:
print(train_data)

In [ ]:
#Get valid indices
import random
random.seed(42) #this seed was used specifically to compare with Okwugbe model



v = 300 #200 samples for valid. Change as you want
test_list = [i for i in range(len(t_data))]
valid_indices = random.choices(test_list, k=v)


test_data = [t_data[i] for i in range(len(t_data)) if i not in valid_indices]
valid_data = [t_data[i] for i in range(len(t_data)) if i in valid_indices]

In [ ]:
def create_json_file(d):
  utterance = d[2]
  wav_path =d[0]
  wav_path = wav_path.replace("/home/ubuntu/organised_recording","/home/ubuntu/LM/LingalaAudio/pyLingala-master")
  return {
      "path": wav_path,
      "sentence": utterance
  }

train_json = [create_json_file(i) for i in train_data]
test_json = [create_json_file(i) for i in test_data]
valid_json = [create_json_file(i) for i in valid_data]

In [ ]:
import os

In [ ]:
#Make folder in GDrive to store files
#train_path = '/content/drive/MyDrive/lingala_xlsr2/train'
#test_path = '/content/drive/MyDrive/lingala_xlsr2/test'
#valid_path = '/content/drive/MyDrive/lingala_xlsr2/valid'

train_path = '/home/ubuntu/LM/lingala_xlsr/train'
test_path = '/home/ubuntu/LM/lingala_xlsr/test'
valid_path = '/home/ubuntu/LM/lingala_xlsr/valid'

if not os.path.isdir(train_path):
  print("Creating paths")
  os.makedirs(train_path)
  os.makedirs(test_path)
  os.makedirs(valid_path)

In [ ]:
import json
#for train
for i, sample in enumerate(train_json):
  file_path = os.path.join(train_path,'train_lingala_{}.json'.format(i))
  with open(file_path, 'w') as outfile:
    json.dump(sample, outfile)

#for test
for i, sample in enumerate(test_json):
  file_path = os.path.join(test_path,'test_lingala_{}.json'.format(i))
  with open(file_path, 'w') as outfile:
    json.dump(sample, outfile)

#for valid
for i, sample in enumerate(valid_json):
  file_path = os.path.join(valid_path,'valid_lingala_{}.json'.format(i))
  with open(file_path, 'w') as outfile:
    json.dump(sample, outfile)

In [ ]:
#run the second time after the error
from datasets import load_dataset, load_metric

#for train
for root, dirs, files in os.walk(train_path):
  lingala_train = load_dataset("json", data_files=[os.path.join(root,i) for i in files],split="train")

#for test
for root, dirs, files in os.walk(test_path):
  lingala_test = load_dataset("json", data_files=[os.path.join(root,i) for i in files],split="train")

#for valid
for root, dirs, files in os.walk(valid_path):
  lingala_valid = load_dataset("json", data_files=[os.path.join(root,i) for i in files],split="train")

In [ ]:
lingala_test

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    print(len(dataset))
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(lingala_test, num_examples=20)

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
lingala_train = lingala_train.map(remove_special_characters)
lingala_test = lingala_test.map(remove_special_characters)
lingala_valid = lingala_valid.map(remove_special_characters)

In [ ]:
show_random_elements(lingala_train.remove_columns(["path"]))

In [ ]:
#def extract_all_chars(batch):
#  all_text = " ".join(batch["sentence"])
#  vocab = list(set(all_text))
#  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
#vocab_train = lingala_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_train.column_names)
#vocab_test = lingala_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_test.column_names)
#vocab_valid = lingala_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=lingala_valid.column_names)

In [ ]:
#vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]) | set(vocab_valid["vocab"][0]))

In [ ]:
#vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [ ]:
#vocab_dict["|"] = vocab_dict[" "]
#del vocab_dict[" "]

In [ ]:
#vocab_dict["[UNK]"] = len(vocab_dict)
#vocab_dict["[PAD]"] = len(vocab_dict)
#len(vocab_dict)

In [ ]:
#from transformers import Wav2Vec2ProcessorWithLM

#processor = Wav2Vec2ProcessorWithLM.from_pretrained("/content/xlsr-lingala1")
#model = Wav2Vec2ForCTC.from_pretrained("/content/xlsr-lingala1")

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
lingala_train = lingala_train.map(speech_file_to_array_fn, remove_columns=lingala_train.column_names)
lingala_test = lingala_test.map(speech_file_to_array_fn, remove_columns=lingala_test.column_names)
lingala_valid = lingala_valid.map(speech_file_to_array_fn, remove_columns=lingala_valid.column_names)

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
lingala_train = lingala_train.map(prepare_dataset, remove_columns=lingala_train.column_names, batch_size=4, num_proc=2, batched=True)
lingala_test = lingala_test.map(prepare_dataset, remove_columns=lingala_test.column_names, batch_size=4, num_proc=2, batched=True)
lingala_valid = lingala_valid.map(prepare_dataset, remove_columns=lingala_valid.column_names, batch_size=4, num_proc=2, batched=True)

## test a single file

In [ ]:
import librosa
import torchaudio

In [ ]:
# load the audio data (use your own wav file here!)
input_audio, sr = librosa.load('/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/test/wav/deborah/220718-132354_lin_9f7_elicit_11.wav', sr=16000)

In [ ]:
# tokenize
input_values = processor(input_audio, return_tensors="pt")

In [ ]:
# retrieve logits
import torch

with torch.no_grad():
  logits = model(**input_values).logits

In [ ]:
# decode using n-gram
transcription = processor.batch_decode(logits.numpy()).text


In [ ]:
transcription[0].lower()

In [ ]:
import IPython.display as ipd

audio_sample = "/home/ubuntu/LM/LingalaAudio/pyLingala-master/data/test/wav/deborah/220718-132354_lin_9f7_elicit_11.wav"
ipd.Audio(data=audio_sample, autoplay=True, rate=audio_sample)

import torch
import torchaudio
import soundfile

waveform, sampling_rate = soundfile.read( audio_sample )

features = processor(waveform, sampling_rate=16000, return_tensors="pt")

#input_values = features.input_values
#attention_mask = features.attention_mask

with torch.no_grad():
  logits = model(**features).logits

transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

In [ ]:
import io
import os
import math
import torch
import warnings
from tqdm.notebook import tqdm


In [ ]:
#input_dict = processor(lingala_test["input_values"][50],sampling_rate=16000, return_tensors="pt", padding=True)

#logits = model(input_dict.input_values).logits

#pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
#logits.shape

In [ ]:
#" ".join(sorted(processor.tokenizer.get_vocab()))


In [ ]:
#transcription = processor.batch_decode(logits.numpy()).text
#transcription[0].lower()


In [ ]:
#print("Prediction:")
#print(processor.decode(pred_ids))

#print(processor.decode(lingala_test["input_values"][0]))

#print("\nReference:")
#print(common_voice_test_transcription["sentence"][0].lower())
#print(processor.decode(lingala_test["labels"][50]))
#print("Target text:", lingala_test[0]["target_text"])

## test on multiple files

In [ ]:
from tqdm import tqdm

In [ ]:
final_pred = []
for i in tqdm(range(lingala_test.shape[0])):    
    input_dict = processor(lingala_test["input_values"][i],sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(input_dict.input_values).logits

    #pred_ids = torch.argmax(logits, dim=-1)[0]
    #prediction = processor.decode(pred_ids)
    prediction = processor.batch_decode(logits.numpy()).text
    final_pred.append(prediction)

In [ ]:
final_pred

In [ ]:
def map_to_pred(batch):
    with torch.no_grad():
      input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
      logits = model(input_values).logits
    #pred_ids = torch.argmax(logits, dim=-1)
    #batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["pred_str"]=processor.batch_decode(logits.numpy()).text[0]
    batch["text"] = processor1.decode(batch["labels"], group_tokens=False)
    return batch

In [ ]:
def map_to_pred(batch):
    with torch.no_grad():
      input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
      logits = model(input_values).logits
    #pred_ids = torch.argmax(logits, dim=-1)
    #batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["pred_str"]=processor.batch_decode(logits.numpy()).text[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
    return batch

In [ ]:
result = lingala_test.map(map_to_pred, remove_columns=lingala_test.column_names)

In [ ]:
result

In [ ]:
show_random_elements(result)

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
#def compute_metrics(pred):
#    pred_logits = pred.predictions
#    pred_ids = np.argmax(pred_logits, axis=-1)
#
#    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
#
#    pred_str = processor.batch_decode(pred_logits.numpy())
    # we do not want to group tokens when computing the metrics
#    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

#    wer = wer_metric.compute(predictions=pred_str, references=label_str)

#    return {"wer": wer}

In [ ]:
from jiwer import wer

In [ ]:
print("Test WER: {:.3f}".format(wer(result["pred_str"], result["text"])))